In [1]:
import requests
import itertools
from arcgis.geometry import Geometry

import pandas as pd
from arcgis.features import SpatialDataFrame as SDF

from arcgis.gis import GIS, Item

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
validate_csv = r'./access_validate_test.csv'

username = 'joel_mccune'
access_item_id = '95c2eceb802c41d8b605b2d431c7547e'

putin_symbol = {"angle":0,"xoffset":12,"yoffset":12,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/GreenFlag.png","contentType":"image/png","width":24,"height":24}
takeout_symbol = {"angle":0,"xoffset":12,"yoffset":12,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/RedFlag.png","contentType":"image/png","width":24,"height":24}
line_symbol = {"type":"esriSLS","style":"esriSLSSolid","color":[0,0,255,255],"width":3}

ldub_reach_id = 2156

In [4]:
class TraceException(Exception):
    pass

def get_epa_point_indexing(x, y, search_distance=5, return_geometry=False):
    
    url = "https://ofmpub.epa.gov/waters10/PointIndexing.Service"
    
    queryString = {
        "pGeometry": "POINT({} {})".format(x, y),
        "pGeometryMod": "WKT,SRSNAME=urn:ogc:def:crs:OGC::CRS84",
        "pPointIndexingMethod": "DISTANCE",
        "pPointIndexingMaxDist": search_distance,
        "pOutputPathFlag": True,
        "pReturnFlowlineGeomFlag": return_geometry,
        "optOutCS": "SRSNAME=urn:ogc:def:crs:OGC::CRS84",
        "optOutPrettyPrint": 0,
        "f": "json"
    }

    r = requests.get( 
        url=url, 
        params=queryString
    )
    
    return r.json()

def get_epa_snap_point(x, y):
    
    rjson = get_epa_point_indexing(x, y)
    
    coords = rjson['output']['end_point']['coordinates']
    
    return {
        "geometry": Geometry(x=coords[0], y=coords[1], spatialReference={"wkid": 4326}),
        "measure": rjson["output"]["ary_flowlines"][0]["fmeasure"],
        "id": rjson["output"]["ary_flowlines"][0]["comid"]
    }

def get_epa_trace_response(putin_point, takeout_point):

    url = "https://ofmpub.epa.gov/waters10/UpstreamDownStream.Service"

    queryString = {
        "pNavigationType": "PP",
        "pStartComID": putin_point["id"],
        "pStartMeasure": putin_point["measure"],
        "pStopComid": takeout_point["id"],
        "pStopMeasure": takeout_point["measure"],
        "pFlowlinelist": True,
        "pTraversalSummary": True,
        "f": "json"
    }

    attempts = 0
    status_code = 0

    while attempts < 10 and status_code != 200:
        resp = requests.get(url, queryString)
        attempts = attempts + 1
        status_code = r.status_code
        if status_code != 200:
            print('Attempt {:02d} failed with status code {}'.format(attempts, status_code))

    return resp

def epa_trace_resp_to_esri_geom(trace_response):
    if trace_response.json()['output']['flowlines_traversed']:
        geom_lists = [feature['shape']['coordinates'] for feature in trace_response.json()['output']['flowlines_traversed']]
        geom_list = list(itertools.chain.from_iterable(geom_lists))
        geom = Geometry({
          "paths" : [geom_list],
          "spatialReference" : {"wkid" : 4326}
        })
        return geom
    else:
        return None
    
def _get_access_from_sdf(sdf, reach_id, access_type):
    
    # just in case the reach_id is provided as an integer
    reach_id = str(reach_id)
    
    # a little error catching
    if access_type != 'putin' and access_type != 'takeout':
        raise TraceException('access_type must be either putin or takeout')
    if type(reach_id) != str:
        raise TraceException('reach_id must be a string representation of an integer')
    
    # get the putin for the reach_id as a SpatialDataFrame
    access_sdf = sdf[(sdf.reach_id == reach_id) & (sdf.type == access_type)]
    
    # if only one record exists, return it - otherwise start breaking stuff
    if len(access_sdf) == 1:
        return access_sdf.iloc[0]
    
    # if there is more than one putin or takeout, blow up
    elif len(access_sdf) > 1:
        raise TraceException('more than one {} exists'.format(access_type, reach_id))
        
    # if no putin or takeout exists, that's bad too
    else:
        raise TraceException('does not have a {}'.format(reach_id, access_type))
    
def get_putin_from_sdf(sdf, reach_id):
    return _get_access_from_sdf(sdf, reach_id, 'putin')

def get_takeout_from_sdf(sdf, reach_id):
    return _get_access_from_sdf(sdf, reach_id, 'takeout')

def get_geometries_from_sdf_by_reach_id(sdf, reach_id):
    
    geom_dict = {
        "putin_point": None,
        "takeout_point": None,
        "reach_line": None
    }
    putin = get_putin_from_sdf(sdf, reach_id)
    takeout = get_takeout_from_sdf(sdf, reach_id)
    
    if not epa_putin:
        raise TraceException('putin cannot be snapped to a hydroline')
    if not epa_takeout:
        raise TraceException('takeout cannot be snapped to a hydroline')

    epa_putin = get_epa_snap_point(putin.SHAPE.x, putin.SHAPE.y)
    epa_takeout = get_epa_snap_point(takeout.SHAPE.x, takeout.SHAPE.y)
    
    geom_dict['putin_point'] = epa_putin['geometry']
    geom_dict['takeout_point'] = epa_takeout['geometry']
    
    epa_trace = get_epa_trace_response(epa_putin, epa_takeout)
    geom_dict['reach_line'] = epa_trace_resp_to_esri_geom(epa_trace)
    
    return geom_dict

def zoom_map_to_geometry(webmap, geom):
    webmap.extent = {
        'type': 'extent',
         'xmin': geom.extent[0],
         'ymin': geom.extent[1],
         'xmax': geom.extent[2],
         'ymax': geom.extent[3],
         'spatialReference': geom.spatial_reference
    }

In [4]:
gis = GIS(username=username)
gis

Enter password: ········


In [5]:
putin_geom = Geometry(y=45.794848, x=-121.634402, spatialReference={'wkid': 4326})
takeout_geom = Geometry(y=45.718817, x=-121.645582, spatialReference={'wkid': 4326})

geom_dict = {
    "error": None,
    "putin_point": None,
    "takeout_point": None,
    "reach_line": None
}

epa_putin = get_epa_snap_point(putin_geom.x, putin_geom.y)
epa_takeout = get_epa_snap_point(takeout_geom.x, takeout_geom.y)

if not epa_putin:
    raise TraceException('putin cannot be snapped to a hydroline')
if not epa_takeout:
    raise TraceException('takeout cannot be snapped to a hydroline')

geom_dict['putin_point'] = epa_putin['geometry']
geom_dict['takeout_point'] = epa_takeout['geometry']

epa_trace = get_epa_trace_response(epa_putin, epa_takeout)
geom_dict['reach_line'] = epa_trace_resp_to_esri_geom(epa_trace)

geom_dict

NameError: name 'r' is not defined

In [88]:
webmap = gis.map()
webmap.basemap = 'national-geographic'
webmap

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

In [89]:
geometries = geom_dict
zoom_map_to_geometry(webmap, geometries['reach_line'])
webmap.clear_graphics()
webmap.draw(geometries['reach_line'], symbol=line_symbol)
webmap.draw(geometries['putin_point'], symbol=putin_symbol)
webmap.draw(geometries['takeout_point'], symbol=takeout_symbol)